In [ ]:
from llama2 import MarketTrendAndInvestmentAdviseToolChain
import asyncio
from dotenv import load_dotenv

load_dotenv(dotenv_path="env")

tool=MarketTrendAndInvestmentAdviseToolChain.from_create(verbose=True)
user_input=input("Your prompt: ")
asyncio.run(tool.arun(user_input))
